In [1]:
import os, getpass
import pandas as pd
from typing import Optional, Dict, Any, Iterable, List

try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    raise ImportError("Could not import sentence_transformers: Please install sentence-transformers package.")
    
try:
    import chromadb
    from chromadb.api.types import EmbeddingFunction
except ImportError:
    raise ImportError("Could not import chromdb: Please install chromadb package.")

c:\Users\firas\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Python program to read
# json file
import json
# Opening JSON file
f = open('./credentials/api.json')
# returns JSON object as
# a dictionary
data = json.load(f)
# Ensure you have your API key set in your environment
#in ./credentials/api.json
IBM_CLOUD_API = data['IBM_CLOUD_API']
PROJECT_ID = data['PROJECT_ID']
# Closing file
f.close()

In [3]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": IBM_CLOUD_API
}

In [4]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = PROJECT_ID

In [5]:
import numpy as np
import pandas as pd

In [6]:
filename_data = "../datacreation/dialogues_embededd.pkl"
data =  pd.read_pickle(filename_data)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_data, test_data= train_test_split(data, test_size=0.05)


In [9]:
train_data.shape


(950, 6)

In [10]:
test_data.shape


(50, 6)

In [11]:
data_root = "../datacreation/"
knowledge_base_dir = f"{data_root}/knowledge_base"

In [12]:
knowledge_base_dir


'../datacreation//knowledge_base'

In [13]:
df = pd.read_csv("../datacreation/dialogues.csv", sep = '\t')
df = df.dropna()#.head(1000)
df.rename(columns = {'Description':'Question',"Doctor":"Answer"}, inplace = True)

#df["combined"] = ("Question: " + df.Question.str.strip() + "\n" +" Patient: " + df.Patient.str.strip()+  "\n" +" Answer: " + df.Answer.str.strip())

df["combined"] = ("Question: " + df.Question.str.strip() + "\n" +" Answer: " + df.Answer.str.strip())

df.head(2)

,Question,Patient,Answer,combined
0,Will masturbation cause weakness in nerves?,"Hi, may I answer your health queries right now...","Hi, Masturbation does make the nerves weak whe...",Question: Will masturbation cause weakness in ...
1,Could lack of hair growth be due to masturbation?,hai sir i am 25years old i used to do masturba...,"Hi, Since you said you masturbate frequently e...",Question: Could lack of hair growth be due to ...


In [14]:
df.shape

(256910, 4)

In [15]:
df.drop_duplicates()
print(f"Original rows: {len(df)}")
df_no_dupes = df.drop_duplicates()
print(f"Rows after removing duplicates: {len(df_no_dupes)}")
print(f"Duplicates removed: {len(df) - len(df_no_dupes)}")

Original rows: 256910
Rows after removing duplicates: 246532
Duplicates removed: 10378


In [16]:
df=df.drop_duplicates()

In [17]:
df.shape

(246532, 4)

In [18]:
df = df.reset_index()
df.rename(columns = {'index':'ids'}, inplace = True)

In [19]:
documents=df


In [20]:
documents.shape


(246532, 5)

In [21]:
documents


,ids,Question,Patient,Answer,combined
0,0,Will masturbation cause weakness in nerves?,"Hi, may I answer your health queries right now...","Hi, Masturbation does make the nerves weak whe...",Question: Will masturbation cause weakness in ...
1,1,Could lack of hair growth be due to masturbation?,hai sir i am 25years old i used to do masturba...,"Hi, Since you said you masturbate frequently e...",Question: Could lack of hair growth be due to ...
2,2,What causes seminal leakage?,"Hi, may I answer your health queries right now...","Hi, These are the side effects of the masturba...",Question: What causes seminal leakage?\n Answe...
3,3,What cause sperm discharge while urinating?,i masturbated too much in my life but now by t...,"Hello, Semen discharge during or after urinati...",Question: What cause sperm discharge while uri...
4,4,How to get rid of semen leakage?,"hi,,,sir im 25 years,,of old,im not married an...","Hello, You have visited many doctors and every...",Question: How to get rid of semen leakage?\n A...
...,...,...,...,...,...
246527,256905,Q. Brother-in-law got a stent for stomach canc...,"Hi doctor, My brother-in-law has stomach cance...",Hi. I understand your concern. For further dou...,Question: Q. Brother-in-law got a stent for st...
246528,256906,Q. Having osteoporosis and spine fracture. How...,"Hi doctor, I have osteoporosis and L3 spine fr...",Hi. I have gone through your message and under...,Question: Q. Having osteoporosis and spine fra...
246529,256907,Q. How long should I take Humira?,"Hi doctor, I am a 55 year old male. I have bee...",Hello. For further information consult a rheum...,Question: Q. How long should I take Humira?\n ...
246530,256908,Q. Tried withdrawal method and took I-pill. Am...,"Hi doctor, I had unprotected sex and took I-pi...",Hi. For further information consult an obstetr...,Question: Q. Tried withdrawal method and took ...


In [22]:
class MiniLML6V2EmbeddingFunction(EmbeddingFunction):
    MODEL = SentenceTransformer('all-MiniLM-L6-v2')
    def __call__(self, texts):
        return MiniLML6V2EmbeddingFunction.MODEL.encode(texts).tolist()
emb_func = MiniLML6V2EmbeddingFunction()

C:\Users\firas\AppData\Local\Temp\ipykernel_23808\1187675410.py:5: DeprecationWarning: The class MiniLML6V2EmbeddingFunction does not implement __init__. This will be required in a future version.
  emb_func = MiniLML6V2EmbeddingFunction()


In [23]:
class ChromaWithUpsert:
    def __init__(
            self,
            name: Optional[str] = "watsonx_rag_collection",
            persist_directory: Optional[str] = None,
            embedding_function: Optional[EmbeddingFunction] = None,
            collection_metadata: Optional[Dict] = None,
    ):
        # Updated client creation approach
        if persist_directory is not None:
            self._client = chromadb.PersistentClient(path=persist_directory)
        else:
            self._client = chromadb.EphemeralClient()
            
        self._embedding_function = embedding_function
        self._persist_directory = persist_directory
        self._name = name
        self._collection = self._client.get_or_create_collection(
            name=self._name,
            embedding_function=self._embedding_function
            if self._embedding_function is not None
            else None,
            metadata=collection_metadata,
        )

    def upsert_texts(
        self,
        texts: Iterable[str],
        metadata: Optional[List[dict]] = None,
        ids: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> List[str]:
        """Run more texts through the embeddings and add to the vectorstore.
        Args:
            :param texts (Iterable[str]): Texts to add to the vectorstore.
            :param metadatas (Optional[List[dict]], optional): Optional list of metadatas.
            :param ids (Optional[List[str]], optional): Optional list of IDs.
            :param metadata: Optional[List[dict]] - optional metadata (such as title, etc.)
        Returns:
            List[str]: List of IDs of the added texts.
        """
        # Convert iterables to lists to work with them more easily
        texts_list = list(texts)
        
        # Generate IDs if not provided
        if ids is None:
            import uuid
            ids = [str(uuid.uuid1()) for _ in texts_list]
        
        # Process in batches of 5000 (safely below the max batch size of 5461)
        batch_size = 5000
        for i in range(0, len(texts_list), batch_size):
            end = min(i + batch_size, len(texts_list))
            
            # Get the current batch
            batch_texts = texts_list[i:end]
            batch_ids = ids[i:end]
            batch_metadata = None if metadata is None else metadata[i:end]
            
            # Upsert the current batch
            self._collection.upsert(
                metadatas=batch_metadata, 
                documents=batch_texts, 
                ids=batch_ids
            )
        
        return ids

    def is_empty(self):
        """Check if the collection is empty."""
        return self._collection.count() == 0

    def persist(self):
        """Persist the collection to disk if using a PersistentClient."""
        # Note: In newer versions of ChromaDB, only PersistentClient needs explicit persist
        if hasattr(self._client, "persist"):
            self._client.persist()

    def query(self, query_texts: str, n_results: int = 5):
        """
        Returns the closests vector to the question vector
        :param query_texts: the question
        :param n_results: number of results to generate
        :return: the closest result to the given question
        """
        return self._collection.query(query_texts=query_texts, n_results=n_results)

In [24]:
%%time
chroma = ChromaWithUpsert(
    name=f"nq910_minilm6v2",
    embedding_function=emb_func,  # you can have something here using /embed endpoint
    persist_directory=knowledge_base_dir,
)
if chroma.is_empty():
    _ = chroma.upsert_texts(
        texts=documents.combined.tolist(),
        # we handle tokenization, embedding, and indexing automatically. 
        #You can skip that and add your own embeddings as well
        metadata=[{'Question': Question,
                   'Patient':Patient,
                   'ids': ids}
                  for (Question,Patient,ids) in
                  zip(documents.Question,documents.Patient, documents.ids)],  # filter on these!
        ids=[str(i) for i in documents.ids],  # unique for each doc
    )
    chroma.persist()

CPU times: total: 391 ms
Wall time: 2.79 s


In [25]:
%%time
chroma = ChromaWithUpsert(
    name=f"nq910_minilm6v2",
    embedding_function=emb_func,  # you can have something here using /embed endpoint
    persist_directory=knowledge_base_dir,
)
if chroma.is_empty():
    _ = chroma.upsert_texts(
        texts=documents.combined.tolist(),
        # we handle tokenization, embedding, and indexing automatically. 
        #You can skip that and add your own embeddings as well
        metadata=[{'Question': Question, 
                   'ids': ids}
                  for (Question,ids) in
                  zip(documents.Question, documents.ids)],  # filter on these!
        ids=[str(i) for i in documents.ids],  # unique for each doc
    )
    chroma.persist()

CPU times: total: 15.6 ms
Wall time: 70.9 ms


In [26]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

In [27]:
model_id = ModelTypes.FLAN_UL2


In [28]:
question_texts = [q.strip("?") + "?" for q in test_data['Question'].tolist()]
print("\n".join(question_texts))

What can cause constant itching around groin area?
What causes pain from anus to testicles?
What causes night fall in men?
What causes testicle pain?
Is masturbating harmful?
What is causing whitish discharge from my penis?
What causes lack of erections in a 17 year old diabetic?
Suggest remedy for pain in my penis?
What causes fishy smelling sperms?
Suggest treatment to increase the hardness of the penis?
Suggest treatment for premature ejaculation?
Can motility and morphology of sperm count be treated by taking paternia?
Suggest treatment for premature ejaculation and erectile dysfunction?
Suggest medication for disorientation and erectile dysfunction of penis?
Suggest remedy for premature ejaculation?
Is loss of virginity related to height growth?
What causes a knot on the testicle?
What causes reduced hardness while on erection?
Suggest remedy for cuts on foreskin near testicles?
How safe is Depsonil and Antoxid for clear fluid leak during masturbation?
How can prostate enlargement

In [29]:
relevant_contexts = []

for question_text in question_texts:
    relevant_chunks = chroma.query(
        query_texts=[question_text],
        n_results=5,
    )
    relevant_contexts.append(relevant_chunks)

In [30]:
sample_chunks = relevant_contexts[0]
for i, chunk in enumerate(sample_chunks['documents'][0]):
    print("=========")
    print("Paragraph index : ", sample_chunks['ids'][0][i])
    print("Paragraph : ", chunk)
    print("Distance : ", sample_chunks['distances'][0][i])

Paragraph index :  584
Paragraph :  Question: What can cause constant itching around groin area?
 Answer: Hello, It is a typical feature of fungal infection (also known as tinea cruris or jock itch). As of now clean the area with soap and water and apply antifungal like clotrimazole for symptomatic relief. An antihistamine like cetrezine can be taken for itching. Good personal hygiene is also important. Generally, the symptoms will settle in a few days. If lesion persists better to consult a dermatologist and get evaluated. Hope I have answered your query. Let me know if I can assist you further. Take care Regards, Dr Shinas Hussain, General & Family Physician
Distance :  0.253711462020874
Paragraph index :  48823
Paragraph :  Question: What causes itching and irritation in groin area?
 Answer: it appears you may have a fungal infection with needs to be examined by a physician to provide further treament
Distance :  0.2962201237678528
Paragraph index :  8634
Paragraph :  Question: What

In [31]:
def make_prompt(context, question_text):
    return (f"Please answer the following.\n"
          + f"{context}:\n\n"
          + f"{question_text}")

prompt_texts = []

for relevant_context, question_text in zip(relevant_contexts, question_texts):
    context = "\n\n\n".join(relevant_context["documents"][0])
    prompt_text = make_prompt(context, question_text)
    prompt_texts.append(prompt_text)

In [32]:
print(prompt_texts[0]) 

Please answer the following.
Question: What can cause constant itching around groin area?
 Answer: Hello, It is a typical feature of fungal infection (also known as tinea cruris or jock itch). As of now clean the area with soap and water and apply antifungal like clotrimazole for symptomatic relief. An antihistamine like cetrezine can be taken for itching. Good personal hygiene is also important. Generally, the symptoms will settle in a few days. If lesion persists better to consult a dermatologist and get evaluated. Hope I have answered your query. Let me know if I can assist you further. Take care Regards, Dr Shinas Hussain, General & Family Physician


Question: What causes itching and irritation in groin area?
 Answer: it appears you may have a fungal infection with needs to be examined by a physician to provide further treament


Question: What causes itching in groin area and discoloration?
 Answer: Thanks for asking in healthcaremagic forumIn short: May be a fungal infectionExpl

In [33]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 200
}

In [34]:
def getBearer(apikey):
    form = {'apikey': apikey, 'grant_type': "urn:ibm:params:oauth:grant-type:apikey"}
    print("About to create bearer")
    # Fix the URL - change iaibmm to iam
    response = requests.post("https://iam.cloud.ibm.com/oidc/token", data=form)
    if response.status_code != 200:
        print("Bad response code retrieving token")
        raise Exception("Failed to get token, invalid status")
    json = response.json()
    if not json:
        print("Invalid/no JSON retrieving token")
        raise Exception("Failed to get token, invalid response")
    print("Bearer retrieved")
    return json.get("access_token")

In [35]:
credentials["token"] = getBearer(credentials["apikey"])

About to create bearer


NameError: name 'requests' is not defined